In [3]:
cd "/Users/waficel-assi/Desktop/StatsMSc/instacart-kaggle/data"

/Users/waficel-assi/Desktop/StatsMSc/instacart-kaggle/data


In [4]:
#import data science libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#data types
import collections

#import general libraries
import gc
import time
import random

#deep learning 
import tensorflow as tf

In [5]:
products = pd.read_csv('products.csv')
print(products.shape)
products.head()

(49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [6]:
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')

In [7]:
orders = pd.read_csv('orders.csv')
print(orders.shape)

(3421083, 7)


In [8]:
orders.head(12)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [9]:
orders_products_prior = pd.read_csv('order_products__prior.csv')
print(orders_products_prior.shape)
orders_products_prior.head(5)

(32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [10]:
orders_products_prior_sample = orders_products_prior[orders_products_prior.order_id <= 1000000]
orders_products_prior_sample.shape

(9473192, 4)

### Merge order_products and orders dataset 

In [11]:
# MEGA DATASET
sample_join_orders = orders_products_prior_sample.set_index('order_id').join(orders.set_index('order_id'), how='left').reset_index()

In [12]:
print(sample_join_orders.shape)
sample_join_orders.head()

(9473192, 10)


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,prior,3,5,9,8.0
1,2,28985,2,1,202279,prior,3,5,9,8.0
2,2,9327,3,0,202279,prior,3,5,9,8.0
3,2,45918,4,1,202279,prior,3,5,9,8.0
4,2,30035,5,0,202279,prior,3,5,9,8.0


In [13]:
last_order_dict = (sample_join_orders
                   .groupby('order_id')
                   .agg({'add_to_cart_order':'max'})
                   .to_dict()['add_to_cart_order'])

In [14]:
sample_join_orders['last_order'] = sample_join_orders['order_id'].map(last_order_dict)
sample_join_orders['last_order_flag'] = (np.where(sample_join_orders['last_order'] == sample_join_orders['add_to_cart_order'],
                                         1,0))

In [15]:
print(sample_join_orders.shape)
sample_join_orders.head(10)

(9473192, 12)


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,last_order,last_order_flag
0,2,33120,1,1,202279,prior,3,5,9,8.0,9,0
1,2,28985,2,1,202279,prior,3,5,9,8.0,9,0
2,2,9327,3,0,202279,prior,3,5,9,8.0,9,0
3,2,45918,4,1,202279,prior,3,5,9,8.0,9,0
4,2,30035,5,0,202279,prior,3,5,9,8.0,9,0
5,2,17794,6,1,202279,prior,3,5,9,8.0,9,0
6,2,40141,7,1,202279,prior,3,5,9,8.0,9,0
7,2,1819,8,1,202279,prior,3,5,9,8.0,9,0
8,2,43668,9,0,202279,prior,3,5,9,8.0,9,1
9,3,33754,1,1,205970,prior,16,5,17,12.0,8,0


In [16]:
orders_products = sample_join_orders.merge(products, on='product_id', how='outer')
orders_products_aisles = orders_products.merge(aisles, on='aisle_id', how='outer')
orders_products_departments = orders_products_aisles.merge(departments, on='department_id', how='outer')

In [17]:
columns_df_product_embeddings = ['user_id','order_id','order_number','add_to_cart_order','product_id','product_name']

In [18]:
sorted_orders_products_departments = orders_products_departments.sort_values(['user_id',
                                                                              'order_number',
                                                                              'order_id',
                                                                              'add_to_cart_order'])

In [19]:
user_id_df = sorted_orders_products_departments[sorted_orders_products_departments.user_id == 10]

In [20]:
sorted_orders_products_departments.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,last_order,last_order_flag,product_name,aisle_id,department_id,aisle,department
7178054,473747.0,196,1.0,1.0,1.0,prior,3.0,3.0,12.0,21.0,5.0,0.0,Soda,77,7,soft drinks,beverages
6250952,473747.0,12427,2.0,1.0,1.0,prior,3.0,3.0,12.0,21.0,5.0,0.0,Original Beef Jerky,23,19,popcorn jerky,snacks
6302341,473747.0,10258,3.0,1.0,1.0,prior,3.0,3.0,12.0,21.0,5.0,0.0,Pistachios,117,19,nuts seeds dried fruit,snacks
1177272,473747.0,25133,4.0,0.0,1.0,prior,3.0,3.0,12.0,21.0,5.0,0.0,Organic String Cheese,21,16,packaged cheese,dairy eggs
4637501,473747.0,30450,5.0,0.0,1.0,prior,3.0,3.0,12.0,21.0,5.0,1.0,Creamy Almond Butter,88,13,spreads,pantry


In [21]:
df_product_embeddings = sorted_orders_products_departments[columns_df_product_embeddings]

In [22]:
df_product_embeddings.head(20)

,user_id,order_id,order_number,add_to_cart_order,product_id,product_name
7178054,1.0,473747.0,3.0,1.0,196,Soda
6250952,1.0,473747.0,3.0,2.0,12427,Original Beef Jerky
6302341,1.0,473747.0,3.0,3.0,10258,Pistachios
1177272,1.0,473747.0,3.0,4.0,25133,Organic String Cheese
4637501,1.0,473747.0,3.0,5.0,30450,Creamy Almond Butter
7177558,1.0,431534.0,5.0,1.0,196,Soda
6250870,1.0,431534.0,5.0,2.0,12427,Original Beef Jerky
6302319,1.0,431534.0,5.0,3.0,10258,Pistachios
1177189,1.0,431534.0,5.0,4.0,25133,Organic String Cheese
4060922,1.0,431534.0,5.0,5.0,10326,Organic Fuji Apples


In [23]:
#list of products in order
list_of_products = df_product_embeddings.product_name.tolist()

In [24]:
#number of unique products in dataset
print(len(set(list_of_products)))
n_products = len(set(list_of_products))

49688


In [25]:
#build skip-gram dataset
def embeddings_pre_processing(categories, n_categories):
    """Process raw inputs into a dataset."""
    
    count = [['UNK', -1]]
    count.extend(collections.Counter(categories).most_common(n_categories - 1))
    most_common_category_dict = dict()
    for category, _ in count:
        most_common_category_dict[category] = len(most_common_category_dict)
    
    idx_categories = list()
    unk_count = 0
    for category in categories:
        if category in most_common_category_dict:
            index = most_common_category_dict[category]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        idx_categories.append(index)
    
    count[0][1] = unk_count
    reversed_most_common_category_dict = dict(zip(most_common_category_dict.values(),
                                                   most_common_category_dict.keys()))
    
    return idx_categories, count, most_common_category_dict, reversed_most_common_category_dict

In [26]:
idx_products, count_products, most_common_products_dict, reversed_most_common_products_dict = embeddings_pre_processing(list_of_products, 20000)

In [27]:
data_index = 0
# generate batch data
def generate_batch(data, batch_size, num_skips, skip_window):
    
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    context = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window input_word skip_window ]
    
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    
    for i in range(batch_size // num_skips):
        target = skip_window  # input word at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]  # this is the input word
            context[i * num_skips + j, 0] = buffer[target]  # these are the context words
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    
    return batch, context


In [28]:
batch_size = 300
embedding_size = 300  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a context.
products_size = 20000

In [29]:
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

In [50]:
tf.reset_default_graph()

In [51]:
tf.get_default_graph()

In [52]:
graph = tf.Graph()

In [53]:
num_sampled=20000

In [54]:
with graph.as_default():
        
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_context = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    embeddings = tf.get_variable('embeddings_graph', [products_size, embedding_size])
    embed_product_ids = tf.nn.embedding_lookup(embeddings, train_inputs)
    
    # Construct the variables for the NCE loss
    nce_weights = tf.get_variable('nce_weights', [products_size, embedding_size])
    nce_biases = tf.get_variable('biases', [products_size], initializer=tf.zeros_initializer)

    nce_loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=train_context,
                       inputs=embed_product_ids,
                       num_sampled=num_sampled,
                       num_classes=products_size))

    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(nce_loss)

    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

    # Add variable initializer.
    init = tf.global_variables_initializer()

In [210]:
# #create embeddings variable and lookup tensor
# embeddings = tf.get_variable('embeddings_y', [20000, embedding_size])
# embed_product_ids = tf.nn.embedding_lookup(embeddings, train_inputs)

# #create weights and biases
# weights = tf.get_variable('weights_embeds', [20000, embedding_size])
# biases = tf.get_variable('biases', [20000], initializer=tf.zeros_initializer)
# hidden_out = tf.matmul(embed_product_ids, tf.transpose(weights)) + biases

# # convert train_context to a one-hot format
# train_one_hot = tf.one_hot(train_labels, 20000)
# cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hidden_out, labels=train_one_hot))
# # Construct the SGD optimizer using a learning rate of 0.1.
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cross_entropy)

# norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
# normalized_embeddings = embeddings / norm
# valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
# similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

# init = tf.global_variables_initializer()


In [55]:
def get_embeddings(graph, num_steps=4000):
    with tf.Session(graph = graph) as session:
    
        # We must initialize all variables before we use them.
        init.run()
        print('Initialized')

        #tf.global_variables_initializer().run()

        average_loss = 0
        for step in range(num_steps):
            batch_inputs, batch_context = generate_batch(idx_products, batch_size, num_skips, skip_window)
            feed_dict = {train_inputs: batch_inputs, train_context: batch_context}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
            _, loss_val = session.run([optimizer, nce_loss], feed_dict=feed_dict)
            average_loss += loss_val

            if step % 2000 == 0:
                if step > 0:
                    average_loss /= 2000
                  # The average loss is an estimate of the loss over the last 2000 batches.
                print('Average loss at step ', step, ': ', average_loss)
                average_loss = 0

        final_embeddings = normalized_embeddings.eval()

In [56]:
get_embeddings(graph)

Initialized
Average loss at step  0 :  13864.1611328125
Average loss at step  2000 :  62.00909757518768
